<a href="https://colab.research.google.com/github/dsibi/yandex_praktikum_da/blob/main/assess_reliability_of_bank_borrowers/assess_reliability_of_bank_borrowers_ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Импортируем библиотекиу pandas для обработки и анализа табличных данных, а также библиотеку pymystem3 для проведения лемматизации

In [ ]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

Читаем и выводим на экран csv-файл

In [ ]:
df = pd.read_csv('data.csv')
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

Файл открыли и изучли - он содержит таблицу, состоящую 21525 строк и 12 колонок:
- не все данные присутствуют в колонках:
    - `days_employed`
    - `total_income`
- колонки с некорректным типом данных:
    - `days_employed`
    - `total_income`
- явные ошибки в данных в колонках `children`, `days_employed`;
- ряд названий колонок нужно изменить.

### Шаг 2. Предобработка данных

### Изменение названий столбцов и обработка пропусков

#### Изменение названий столбцов

Выводим текущие названия столбцов

In [ ]:
df.columns

NameError: ignored

Корректируем названия столбцов

In [ ]:
df.rename(columns={'children': 'children_number', 'dob_years': 'age_years', 'education': 'education_level', 'education_id': 'education_level_id', 'purpose': 'loan_purpose'}, inplace=True)
df.columns

NameError: ignored

#### Обработка пропусков

##### Считаем доли пропусков

In [ ]:
df.isna().mean()

Доля пропусков по следующим колонкам `days_employed` и `total_income` одинакова и равна 10 %.
Возможные причины появления пропусков - технические проблемы при выгрузке из разных баз данных.

##### Заполненияем пропуски

- `days_employed`: данные по этому столбцу не фигурируют в запросе Заказчика, поэтому пропуски заменяем нулями

In [ ]:
df['days_employed']=df['days_employed'].fillna(0)

- `total_income` заполним медианным значением в соответствии с типом занятости, для этого производим:

    - расчет медианного значения по `income_type` и сохранение его в `med_table`

In [ ]:
med_table=df.groupby('income_type')['total_income'].median()

    - подставновку рассчитанного медианного значения в пропущенные ячейки

In [ ]:
for income_type in med_table.index:
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income_type), 'total_income'] = med_table.loc[income_type]

    - проверяем наличие пропусков

In [ ]:
df.info()

### Вывод

Скорректировали наименования полей таблицы и заполнили все пропуски в ее ячейках.

### Замена типа данных

Заменяем вещественный тип данных на целочисленный в столбцах `days_employed` и `total_income` и производим проверку изменений

In [ ]:
for col in ['days_employed', 'total_income']:
    df[col]=df[col].astype('int')
df.dtypes

children_number        int64
days_employed          int64
age_years              int64
education_level       object
education_level_id     int64
family_status         object
family_status_id       int64
gender                object
income_type           object
debt                   int64
total_income           int64
loan_purpose          object
dtype: object

### Вывод

Тип данных изменен. Метод для их изменения был выбран в соответствии с начальным типом данных - float64.

### Обработка дубликатов

Для более эффективного поиска дубликатов приведем данные в таблице к нижнему регистру с применением конструкции try-except, чтобы исключить возможные ошибки отсутствия атрибута str.lower в колнках таблицы - AttributeError

In [ ]:
for col in df.columns:
    try:
        df[col] = df[col].str.lower()
    except AttributeError:
        pass

Подсчитаем количество дубликатов в таблице

In [ ]:
print('Дубликатов в таблице: ', df.duplicated().sum())

Дубликатов в таблице:  71


Удаляем дубликаты с помощью метода `drop_duplicates`, оставляя первую строку из перечня дублей и производим проверку

In [ ]:
df=df.drop_duplicates()
print('Дубликатов в таблице: ', df.duplicated().sum())

Дубликатов в таблице:  0


### Вывод

Из таблицы удален 71 дубликат.
Возможные причины появления дубликатов - технические проблемы при выгрузке из разных баз данных.

### Лемматизация

Создаем функцию для лемматизация текста

In [ ]:
def lemmatization (loan_purpose):
    lemmas = ' '.join(m.lemmatize(loan_purpose))
    if 'автомобиль' in lemmas:
        return 'покупка автомобиля'
    elif ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'покупка недвижимости'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return 'другая категория'

Применяем созданную функцию к колонке `loan_purpose` и добавляем колонку с леммами - `loan_purpose_dict`

In [ ]:
df['loan_purpose_dict']=df['loan_purpose'].apply(lemmatization)

Выводим перечень целей получения кредита, отсортированный по убыванию

In [ ]:
loan_purposes=df.groupby('loan_purpose_dict')['loan_purpose_dict'].count()
print(loan_purposes.sort_values(ascending = False))

loan_purpose_dict
покупка недвижимости    10811
покупка автомобиля       4306
образование              4013
свадьба                  2324
Name: loan_purpose_dict, dtype: int64


### Вывод

Метод лемматизации определил, что около 50 % клиентов берут кредит на приобретение недвижимости.

### Категоризация данных

При работе с набором данных были созданы и использованы следующие «словари»:
- `columns` - для переименования названий колонок;
- `med_table` - для подставновки рассчитанного медианного значения в пропущенные ячейки колонки `total_income`;
- `{-1 : 1, 20 : 2}` - для корректировки значений в колонке `children_number`.

In [ ]:
df['children_number'] = df['children_number'].replace({-1 : 1, 20 : 2})
df.children_number.value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children_number, dtype: int64

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
children_debt=df.pivot_table(index='children_number',values='debt',aggfunc=['count','sum'])
children_debt['%']=round((children_debt[('sum', 'debt')]/children_debt[('count', 'debt')])*100,2)
children_debt.columns = ['Общее количество заемщиков','Количество просрочников','%']
children_debt

,Общее количество заемщиков,Количество просрочников,%
children_number,,,
0,14091,1063,7.54
1,4855,445,9.17
2,2128,202,9.49
3,330,27,8.18
4,41,4,9.76
5,9,0,0.00


### Вывод

Несмотря на максимальное количество клиентов с просроченными кредитами у которых нет детей, доля их в сравнении с иными категориями клиентов меньшая и составляет 7,54 %, в то время как доли просроченных кредитов у клиентов имеющих от 1 до 4 детей - от 8,18 % до 9,76 %. В силу малого количества попавших в выборку клиентов с 5-ю детьми, данные по ним можно нивелировать.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_status_debt=df.pivot_table(index='family_status', values='debt',aggfunc=['count','sum'])
family_status_debt['%']=round((family_status_debt[('sum', 'debt')]/family_status_debt[('count', 'debt')])*100,2)
family_status_debt.columns = ['Общее количество заемщиков','Количество просрочников','%']
family_status_debt.sort_values(by = '%', ascending = False)

,Общее количество заемщиков,Количество просрочников,%
family_status,,,
не женат / не замужем,2810,274,9.75
гражданский брак,4151,388,9.35
женат / замужем,12339,931,7.55
в разводе,1195,85,7.11
вдовец / вдова,959,63,6.57


### Вывод

В большей степени просрочку допускают заемщики, не обремененные семейными узами - 9,75 % от общего количества всех не женатых / не замужних, но также недалеко отстают от них лица, живущие в гражданском браке - 9,35 % от общего числа состоящих в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
import numpy as np
total_income_debt=pd.DataFrame(df, columns=['total_income', 'debt'])
total_income_debt['income_groups'] = pd.cut(total_income_debt['total_income'],[0,50000,100000,150000,200000,np.inf],labels=["до 50 тыс. руб.", "50-100 тыс. руб.", "100-150 тыс. руб.","150-200 тыс. руб.","свыше 200 тыс. руб."])
total_income_debt=total_income_debt.groupby('income_groups')['debt'].agg(['count','sum'])
total_income_debt['%']=round((total_income_debt[('sum')]/total_income_debt[('count')])*100,2)
total_income_debt.columns = ['Общее количество заемщиков','Количество просрочников','%']
total_income_debt

,Общее количество заемщиков,Количество просрочников,%
income_groups,,,
до 50 тыс. руб.,372,23,6.18
50-100 тыс. руб.,4091,331,8.09
100-150 тыс. руб.,7160,624,8.72
150-200 тыс. руб.,4764,405,8.50
свыше 200 тыс. руб.,5067,358,7.07


### Вывод

Прямая зависимость между уровнем дохода и наличием просроченной задолженности не наблюдается и незначительно колеблется в диапазоне от 6,18 % до 8,7 %.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
loan_purpose_debt=pd.DataFrame(df, columns=['loan_purpose_dict', 'debt'])
loan_purpose_debt_gr=loan_purpose_debt.groupby('loan_purpose_dict')['debt'].agg(['count','sum'])
loan_purpose_debt_gr['%']=round((loan_purpose_debt_gr[('sum')]/loan_purpose_debt_gr[('count')])*100,2)
loan_purpose_debt_gr.columns = ['Общее количество заемщиков','Количество просрочников','%']
loan_purpose_debt_gr.sort_values(by='%',ascending = False)

,Общее количество заемщиков,Количество просрочников,%
loan_purpose_dict,,,
покупка автомобиля,4306,403,9.36
образование,4013,370,9.22
свадьба,2324,186,8.00
покупка недвижимости,10811,782,7.23


### Вывод

Максимальный процент просрочки по выданным кредитам наблюдается по автокредитам и составляет 9,36 %, минимальный же по ипотеке – 7,23 %.

### Шаг 4. Общий вывод

In [ ]:
print("Средняя просрочка по кредитному портфелю: %.2f" % ((df['debt'].sum()/len(df)*100)), '%')

Средняя просрочка по кредитному портфелю: 8.12 %


В целях снижения уровня просроченных кредитов рекомендуем Заказчику пересмотреть скоринговую политику в отношении следующих групп клиентов:
- клиентов с детьми;
- не женатых/холостых, а также клиентов, проживающих в гражданском браке;
- клиентов, подающих заявку на автокредитование.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.